In [1]:
import numpy as np
import textblob as textblob
import praw
from datetime import datetime
import matplotlib.pyplot as plt
import json
import creds
from pprint import pprint
import pandas as pd
from time import time

In [2]:
#Importing NLTK library and associated packaged

import nltk
nltk.__version__
# nltk.download('vader_lexicon')
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
from nltk.tag import pos_tag_sents
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

#Importing textblob to compare sentiment analysis results with those from nltk

from textblob import TextBlob

# Functions

In [ ]:
#pulls all discussion ids available and adds

def get_disc_ids(subreddit, disc_dict):
    
    discussion_ids = []
    for submission in subreddit.search('Daily Discussion Post'):
        if 'Daily Discussion Post' in submission.title:
            discussion_ids.append(submission.id)
      
    for d in discussion_ids:
        if d not in disc_dict.keys():
            print("Added", d, "to discussion dictionary.")
            disc_dict[d] = {}
        else: 
            pass
    
    if len(discussion_ids) != len(disc_dict):
        print("Alert! Length of discussion_ids does not equal length of dictionary.")
    
    return disc_dict

In [4]:
def todays_disc_id(NAME_OF_SUBREDDIT):
    subreddit = reddit.subreddit(NAME_OF_SUBREDDIT)
    discussion_id = str(list(subreddit.hot(limit=1)))
    final_id = discussion_id.replace("[Submission(id='","").replace("')]","")
    return(final_id)

In [10]:
today = [todays_disc_id('Coronavirus')]
today

['gibmed']

In [6]:
empty = {'placeholder':{}}

In [21]:
def comment_info(us_com, submission, sid):
    
    token_dict = {}
    
    comment = us_com.body
    comment = comment.replace('\n', ' ')
    comment = comment.replace('I\'m', 'i am').replace('i\'m', 'i am').replace('i\'ll', 'i will').replace('I\'ll', 'i will')
    comment = comment.lower()
    
    #grab date/ time info for each comment 
    utc = submission.created_utc
    dt_object = datetime.fromtimestamp(utc)  

    #performing sentiment analysis
    ss = sid.polarity_scores(comment)
    
    if comment != '[removed]':
        token_dict.update({"comment_body" : comment})
        token_dict.update({"month":dt_object.strftime("%B")})
        token_dict.update({"day" : dt_object.strftime("%d")})
        token_dict.update(ss)
    
    return token_dict


In [22]:
def subreddit_sentiment(discussion_ids, existing_dict, reddit = reddit):

    sid = SentimentIntensityAnalyzer()
    st = time()
    dictionary = {}
    
    for i in discussion_ids:
        print(i)
        dictionary[i] = {}
        submission = reddit.submission(i)
        submission.comments.replace_more(limit=0)

        for user_comment in submission.comments:
            tok_dict = comment_info(user_comment, submission, sid)

            if str(user_comment) not in existing_dict.keys():
                dictionary[i].setdefault(str(user_comment),tok_dict)
    
    
    print("\nProcessing time:", round((time()-st)/60, 2), "minutes.")
    
    return dictionary

In [23]:
testing = subreddit_sentiment(today, empty)

gibmed

Processing time: 0.11 minutes.


In [24]:
testing

{'gibmed': {'fqe8a44': {'comment_body': 'this article about indoor vs. outdoor exposure is getting a lot of traction:\xa0[https://www.erinbromage.com/post/the-risks-know-them-avoid-them](https://www.erinbromage.com/post/the-risks-know-them-avoid-them)  it’s not a peer reviewed paper but does seem to summarize several around mers and sars. gist is that outdoor spread is quite rare, you seldom get enough exposure to get infected. walking into the street to avoid someone on the sidewalk might have no purpose.  on the other hand, certain indoor spaces are really good at spreading the virus. you’re just getting so much exposure to virus that way. even worse if someone coughs or sneezes. if this author ends up being right, i’m not seeing how it would be safe to be in an office environment or indoor space for very long with people who could have the virus. (vs. a grocery store with a mask where you don’t seem to get enough of a dose in that short amount of time).  **"remember the formula: suc

In [ ]:
def get_disc_comments(reddit, disc_id):

    num = 0
    sid = SentimentIntensityAnalyzer()
    st = time()
    
    for k in disc_id.keys():
        num += 1
        print("Pulling comments for discussion", num, "of", len(disc_id))
        submission = reddit.submission(k)
        submission.comments.replace_more(limit=0)
        
        for user_comment in submission.comments:
            tok_dict = comment_info(user_comment, submission, sid)
            
            if str(user_comment) not in disc_id[k].keys():
                disc_id[k].setdefault(str(user_comment),tok_dict)
        
    print("\nProcessing time:", round((time()-st)/60, 2), "minutes.")
    return disc_id
        

In [ ]:
def add_new_comments(json_file_path, reddit, subreddit):
    
    with open(json_file_path, "r") as f:
        full_comment_dict = json.load(f)
    
        discussion_ids = get_disc_ids(subreddit, full_comment_dict)
        
        all_comment_dict = get_disc_comments(reddit, discussion_ids)
        
        
    #update json file with new comments
    with open(json_file_path, "w") as outfile:
        json.dump(all_comment_dict, outfile, indent = 4)
        
    return all_comment_dict

In [ ]:
json_file_path = r"/Users/AllysonEnglish/Active/Active/coronavirus_subreddit_full.json"

In [3]:
# Reddit API wrapper 

reddit = praw.Reddit(client_id=creds.client_id, \
                     client_secret=creds.client_secret, \
                     user_agent=creds.user_agent, \
                     username=creds.username, \
                     password=creds.password)

#Getting daily ID for the Coronavirus daily discussion post

subreddit = reddit.subreddit('Coronavirus')

# updated_dict = add_new_comments(json_file_path, reddit, subreddit)

In [ ]:
def subreddit_sentiment(discussion_ids, existing_dict, reddit = reddit):

    sid = SentimentIntensityAnalyzer()
    st = time()
    dictionary = {}
    
    for i in discussion_ids:
        print(i)
        dictionary[i] = {}
        submission = reddit.submission(i)
        submission.comments.replace_more(limit=0)

        for user_comment in submission.comments:
            tok_dict = comment_info(user_comment, submission, sid)

            if str(user_comment) not in existing_dict.keys():
                dictionary[i].setdefault(str(user_comment),tok_dict)
    
    
    print("\nProcessing time:", round((time()-st)/60, 2), "minutes.")
    
    return dictionary

In [ ]:
g = 0
submission = reddit.submission('gibmed')
submission.comments.replace_more(limit=0)
for user_comment in submission.comments:
        g+=1
g

In [ ]:
test

In [ ]:
this = ['cats', 'bats', 'dogs']

In [ ]:
for t in this:
    print(t)

In [ ]:
test = subreddit_sentiment(today, empty)

In [ ]:
r = 0

for k in test.keys():
    for a in test[k].keys():
        r+=1
r

# Data Analysis

In [ ]:
discussion_id = updated_dict.keys()

m_date = []
d_date = []
negative = []
neutral = []
positive = []
compound = []
comment_id = []
discussion = []

for k in updated_dict.keys():
    
    for y in updated_dict[k].keys():
        discussion.append(k)
        comment_id.append(y)
    
    for x in updated_dict[k]:
        m_date.append(updated_dict[k][x].get('month'))
        d_date.append(updated_dict[k][x].get('day'))
        negative.append(updated_dict[k][x].get('neg'))
        neutral.append(updated_dict[k][x].get('neu'))
        positive.append(updated_dict[k][x].get('pos'))
        compound.append(updated_dict[k][x].get('compound'))

    
d = {'discussion_id' : discussion,'comment_id' : comment_id, 'month': m_date, 'date': d_date, 'positive': positive, 'neutral': neutral, 'negative': negative, 'compound': compound}
df = pd.DataFrame(data=d)
df.head()

In [ ]:
len(df['discussion_id'].unique())

In [ ]:
df['DOM'] = df['month']+df['date']
new_df = df.groupby('DOM').mean()

In [ ]:
avrg_by_day

In [ ]:
avrg_by_day = df.groupby('DOM').mean().reset_index()

def sort_days(x):
    if x.startswith('F'):
        return 2
    elif x.startswith('Mar'):
        return 3
    elif x.startswith('Ap'):
        return 4
    elif x.startswith('May'):
        return 5

avrg_by_day['month_num'] = avrg_by_day['DOM'].apply(sort_days) 

def day_num(x):
    return int(x[-2:])

avrg_by_day['day_num'] = avrg_by_day['DOM'].apply(day_num)

avrg_by_day.sort_values(['month_num', 'day_num'], inplace = True)
avrg_by_day.reset_index(drop=True, inplace= True)

In [ ]:
df['month_num'] = df['DOM'].apply(sort_days)
df['day_num'] = df['DOM'].apply(day_num)
df.sort_values(['month_num', 'day_num'], inplace = True)

In [ ]:
new_df = new_df.reset_index()

In [ ]:
df['negative'].mean(), df['positive'].mean()

In [ ]:
avrg_by_day.head()

In [ ]:
# Graphing 

plt.plot(avrg_by_day.index, avrg_by_day['compound'])
plt.title('Sentiment Average by Day')
plt.ylabel('Sentiment');
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.bar(avrg_by_day.index, avrg_by_day['compound'])

In [ ]:
len(avrg_by_day['DOM'])

In [ ]:
len(updated_dict.keys())